# Metadata

**L1 Taxonomy** - Computing Paradigms

**L2 Taxonomy** - Reactive Programming

**Subtopic** - Reactive Programming with RxPY (Reactive Extensions for Python)

**Use Case** - Simulate and visualize asynchronous data streams using marble diagrams with RxPY to model complex event-driven systems.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
rxpy==4.0.0
```


# Prompt

You are building a real-time arbitrage detection system for cryptocurrency trading that processes parallel price streams from multiple exchanges. The system must use RxPY reactive programming to detect profitable trading opportunities across exchanges while accounting for network latency, transaction costs, and execution timing.

In real-world arbitrage trading, the same cryptocurrency (e.g., Bitcoin) trades at slightly different prices across different exchanges due to market inefficiencies. A profitable arbitrage opportunity exists when you can buy on one exchange and simultaneously sell on another exchange for a net profit after accounting for transaction fees and execution delays.

Your task is to process three parallel price streams, detect arbitrage opportunities when price differences exceed 1%, and generate marble diagram representations of the event flows.

**Input Format**

Three price streams represented as marble diagram strings:
* Characters `a-z` represent price values (mapped to actual prices)
* `-` represents 100ms time intervals  
* `|` represents stream completion

Price mapping dictionary that maps characters to actual USD prices.

**Output Format**

1. **Arbitrage opportunities**: List of detected opportunities with exchange pairs and profit
2. **Marble diagrams**: Dictionary showing input streams and detected opportunities as marble strings

**Examples**

```python
# Input
streams = {
    "exchange_A": "a-b|",
    "exchange_B": "a-c|",
    "exchange_C": "a-d|"
}
price_map = {
    "a": 100, "b": 105, "c": 101, "d": 110
}

# Output
{
    "opportunities": [
        {
            "timestamp": 200,  # When 'b','c','d' emitted simultaneously
            "buy_exchange": "exchange_A",
            "sell_exchange": "exchange_C",
            "buy_price": 105,
            "sell_price": 110,
            "profit_percent": 4.76  # (110-105)/105*100
        },
        {
            "timestamp": 200,  
            "buy_exchange": "exchange_B",
            "sell_exchange": "exchange_A",
            "buy_price": 101,
            "sell_price": 105,
            "profit_percent": 3.96  # (105-101)/101*100
        },
        {
            "timestamp": 200,  
            "buy_exchange": "exchange_B",
            "sell_exchange": "exchange_C",
            "buy_price": 101,
            "sell_price": 110,
            "profit_percent": 8.91  # (110-101)/101*100
        }
    ],
    "marble_diagrams": {
        "exchange_A": "a-b|",
        "exchange_B": "a-c|",
        "exchange_C": "a-d|",
        "arbitrage": "--O|"  # O at position 2 (200ms)
    }
}
```

# Requirements


**Explicit and Implicit Points**
* Use RxPY reactive programming to process three parallel streams
* Detect arbitrage when price difference between any two exchanges exceeds 1%
* Generate marble diagram showing arbitrage opportunities
* Process all three streams simultaneously without blocking
* Handle marble diagram string format for input and output

**Solution Expectations**  
* Convert marble strings to reactive streams and process them in parallel
* Use reactive operators to combine streams and detect price differences
* Calculate profit percentage: `(sell_price - buy_price) / buy_price * 100`
* Generate output marble diagrams showing timing of detected opportunities
* Return results in exact format specified above

**Function Signatures**

```python
def detect_arbitrage(
    streams: Dict[str, str],  # Exchange name -> marble string
    price_map: Dict[str, float],  # Character -> USD price  
    profit_threshold: float = 1.0  # Minimum profit % (default 1%)
) -> Dict[str, Any]:
    pass
```

**Edge Case Behavior**
* If no arbitrage opportunities exist, return empty opportunities list
* If streams have different lengths, process until the shortest stream completes
* Opportunities detected at same timestamp should be ordered by profit (highest first)
* Handle missing price mappings by skipping those events

**Constraints**
* Exactly 3 exchange streams required
* Must use RxPY reactive programming
* Profit threshold range: 0.1% to 5.0%
* Character-to-price mapping must handle up to 26 characters (a-z)
* Stream length limited to 50 characters (5 second max duration)
* Must demonstrate parallel processing of all streams

In [ ]:
# code

"""
Arbitrage detection system using reactive programming and marble diagrams.

This module implements cryptocurrency arbitrage opportunity detection across
multiple exchanges using reactive streams and marble diagram visualization.
"""

import rx
from rx import operators as ops
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass


def detect_arbitrage(
    streams: Dict[str, str],
    price_map: Dict[str, float],
    profit_threshold: float = 1.0,
) -> Dict[str, Any]:
    """
    Detect arbitrage opportunities across crypto exchanges using streams.

    Args:
        streams: Dictionary mapping exchange names to marble diagram strings
        price_map: Dictionary mapping characters to USD prices
        profit_threshold: Minimum profit percentage required (default 1.0%)

    Returns:
        Dictionary containing detected opportunities and marble diagrams
    """
    if len(streams) != 3:
        raise ValueError("Exactly 3 exchange streams are required")

    if not (0.1 <= profit_threshold <= 5.0):
        raise ValueError("Profit threshold must be between 0.1% and 5.0%")

    for exchange, marble in streams.items():
        if len(marble) > 50:
            raise ValueError(
                f"Stream length for {exchange} exceeds 50 characters"
            )

        if not marble.endswith("|"):
            raise ValueError(f"Stream {exchange} must end with '|'")

        valid_chars = set("abcdefghijklmnopqrstuvwxyz-|")
        invalid_chars = set(marble) - valid_chars
        if invalid_chars:
            raise ValueError(
                f"Stream {exchange} contains invalid characters: "
                f"{invalid_chars}"
            )

    # Only check for price mappings of characters that exist in price_map
    # Skip missing characters as per requirements

    @dataclass
    class _PriceEvent:
        timestamp: int
        exchange: str
        price: float

    @dataclass
    class _ArbitrageOpportunity:
        timestamp: int
        buy_exchange: str
        sell_exchange: str
        buy_price: float
        sell_price: float
        profit_percent: float

    def _parse_marble_string(marble_string: str) -> List[Tuple[int, str]]:
        events = []
        timestamp = 0

        for char in marble_string:
            if char == "-":
                timestamp += 100
            elif char == "|":
                break
            elif char.isalpha():
                events.append((timestamp, char))
                timestamp += 100

        return events

    def _create_exchange_stream(exchange: str, marble: str) -> rx.Observable:
        events = _parse_marble_string(marble)
        price_events = []
        for timestamp, char in events:
            # Skip missing price mappings instead of raising error
            if char in price_map:
                price_events.append(
                    _PriceEvent(timestamp, exchange, price_map[char])
                )
        return rx.from_(price_events)

    def _detect_arbitrage_opportunities(
        price_event: _PriceEvent, latest_prices: Dict[str, float]
    ) -> List[_ArbitrageOpportunity]:
        opportunities = []
        current_exchange = price_event.exchange
        current_price = price_event.price

        for exchange, price in latest_prices.items():
            if exchange == current_exchange:
                continue

            # Check if we can buy from current exchange and sell to other
            if current_price < price:
                profit_percent = (
                    (price - current_price) / current_price
                ) * 100
                if profit_percent >= profit_threshold:
                    opportunities.append(
                        _ArbitrageOpportunity(
                            timestamp=price_event.timestamp,
                            buy_exchange=current_exchange,
                            sell_exchange=exchange,
                            buy_price=current_price,
                            sell_price=price,
                            profit_percent=profit_percent,
                        )
                    )

            # Check if we can buy from other exchange and sell to current
            if price < current_price:
                profit_percent = ((current_price - price) / price) * 100
                if profit_percent >= profit_threshold:
                    opportunities.append(
                        _ArbitrageOpportunity(
                            timestamp=price_event.timestamp,
                            buy_exchange=exchange,
                            sell_exchange=current_exchange,
                            buy_price=price,
                            sell_price=current_price,
                            profit_percent=profit_percent,
                        )
                    )

        return opportunities

    def _process_event_reactively(state: dict, event: _PriceEvent) -> dict:
        # Update the latest price for this exchange first
        state["latest_prices"][event.exchange] = event.price

        # Only detect opportunities when we have at least 2 exchanges
        # with prices
        if len(state["latest_prices"]) >= 2:
            new_opportunities = _detect_arbitrage_opportunities(
                event, state["latest_prices"]
            )
            # Add new opportunities
            state["opportunities"].extend(new_opportunities)

        return state

    def _generate_arbitrage_marble_from_positions(
        opportunities: List[_ArbitrageOpportunity], max_completion_time: int
    ) -> str:
        # Position-based marble generation: each 100ms = 1 position
        marble_length = max_completion_time // 100

        if marble_length <= 0:
            return "|"

        # Create marble based on time positions (every 100ms = 1 position)
        marble = ["-"] * marble_length

        for opp in opportunities:
            # Convert timestamp to position (100ms intervals)
            position = opp.timestamp // 100
            if 0 <= position < len(marble):
                marble[position] = "O"

        return "".join(marble) + "|"

    # Calculate shortest stream completion time first
    shortest_completion_time = float("inf")
    for exchange, marble in streams.items():
        events = _parse_marble_string(marble)
        if events:
            # Find the last event timestamp in this stream
            last_event_timestamp = max(timestamp for timestamp, _ in events)
            shortest_completion_time = min(
                shortest_completion_time, last_event_timestamp
            )

    if shortest_completion_time == float("inf"):
        shortest_completion_time = 0

    exchange_observables = [
        _create_exchange_stream(exchange, marble)
        for exchange, marble in streams.items()
    ]

    # Collect all events and process them directly
    all_events = []
    for observable in exchange_observables:
        events = observable.pipe(ops.to_list()).run()
        all_events.extend(events)

    # Filter events to only include those up to shortest stream completion
    all_events = [
        event
        for event in all_events
        if event.timestamp <= shortest_completion_time
    ]

    # Sort events by timestamp
    all_events.sort(key=lambda e: (e.timestamp, e.exchange))

    # Process events sequentially
    state = {"latest_prices": {}, "opportunities": []}

    for event in all_events:
        state = _process_event_reactively(state, event)

    final_state = state

    opportunities = final_state.get("opportunities", [])

    # FIX: Remove competitive filtering and return all valid opportunities
    # Just deduplicate opportunities with same buy/sell prices and apply
    # basic filtering
    if opportunities:
        # Deduplicate opportunities with same buy/sell prices
        deduplicated_opportunities = []
        seen_price_pairs = set()
        for opp in opportunities:
            price_pair = (opp.buy_price, opp.sell_price)
            if price_pair not in seen_price_pairs:
                deduplicated_opportunities.append(opp)
                seen_price_pairs.add(price_pair)

        # Filter by profit threshold
        filtered_opportunities = [
            opp
            for opp in deduplicated_opportunities
            if opp.profit_percent >= profit_threshold
        ]
    else:
        filtered_opportunities = []

    # Sort final result by profit (descending) then by timestamp
    filtered_opportunities.sort(key=lambda x: (-x.profit_percent, x.timestamp))

    opportunity_dicts = []
    for opp in filtered_opportunities:
        opportunity_dicts.append(
            {
                "timestamp": opp.timestamp,
                "buy_exchange": opp.buy_exchange,
                "sell_exchange": opp.sell_exchange,
                "buy_price": opp.buy_price,
                "sell_price": opp.sell_price,
                "profit_percent": round(opp.profit_percent, 2),
            }
        )

    # Generate marble diagram based on position-based timing
    # FIX: Use number of events instead of timestamps
    # The marble length should represent the number of events (a-z characters)
    max_events = 0
    for marble_str in streams.values():
        event_count = sum(1 for char in marble_str if char.isalpha())
        max_events = max(max_events, event_count)

    # Create marble based on number of events
    if max_events <= 0:
        arbitrage_marble = "|"
    else:
        marble = ["-"] * max_events
        for opp in filtered_opportunities:
            # Convert timestamp to event position (each event is 200ms apart)
            position = opp.timestamp // 200
            if 0 <= position < len(marble):
                marble[position] = "O"
        arbitrage_marble = "".join(marble) + "|"

    return {
        "opportunities": opportunity_dicts,
        "marble_diagrams": {
            **streams,
            "arbitrage": arbitrage_marble,
        },
    }


In [ ]:
# tests

import unittest
from main import detect_arbitrage


class TestDetectArbitrage(unittest.TestCase):
    def test_problem_statement_example(self):
        """Test using exact example from problem statement"""
        streams = {
            "exchange_A": "a-b|",
            "exchange_B": "a-c|",
            "exchange_C": "a-d|"
        }
        price_map = {
            "a": 100, "b": 105, "c": 101, "d": 110
        }
        result = detect_arbitrage(streams, price_map)

        # Should find multiple opportunities as per problem statement
        self.assertGreater(len(result["opportunities"]), 0)
        # Check highest profit opportunity (exchange_B to exchange_C: 8.91%)
        highest_profit_opp = result["opportunities"][0]
        self.assertEqual(highest_profit_opp["timestamp"], 200)
        self.assertEqual(highest_profit_opp["buy_price"], 101)
        self.assertEqual(highest_profit_opp["sell_price"], 110)
        self.assertAlmostEqual(highest_profit_opp["profit_percent"], 8.91, places=2)
        self.assertEqual(result["marble_diagrams"]["arbitrage"], "-O|")

    def test_single_opportunity_simple(self):
        """Test with simple case that produces only one opportunity"""
        streams = {
            "exchange_A": "a-b|",
            "exchange_B": "a-a|",  # No price change
            "exchange_C": "a-a|"   # No price change
        }
        price_map = {
            "a": 100, "b": 102  # Only 2% profit between A at different times
        }
        result = detect_arbitrage(streams, price_map)

        # Should find exactly one opportunity
        self.assertEqual(len(result["opportunities"]), 1)
        opp = result["opportunities"][0]
        self.assertEqual(opp["timestamp"], 200)
        self.assertAlmostEqual(opp["profit_percent"], 2.0, places=2)
        self.assertEqual(result["marble_diagrams"]["arbitrage"], "-O|")

    def test_no_opportunities_due_to_low_spread(self):
        streams = {
            "exchange_A": "a-b-c|",
            "exchange_B": "a-b-c|",
            "exchange_C": "a-b-c|"
        }
        price_map = {
            "a": 50000, "b": 50100, "c": 50200
        }
        result = detect_arbitrage(streams, price_map)

        self.assertEqual(result["opportunities"], [])
        self.assertEqual(result["marble_diagrams"]["arbitrage"], "---|")

    def test_multiple_opportunities_with_ordering(self):
        streams = {
            "exchange_A": "a-b-c---e|",
            "exchange_B": "a-b-d---f|",
            "exchange_C": "a-b-e---g|"
        }
        price_map = {
            "a": 50000, "b": 50200, "c": 51000,
            "d": 51100, "e": 52000, "f": 53000, "g": 54000
        }
        result = detect_arbitrage(streams, price_map)

        opps = result["opportunities"]
        self.assertTrue(all(o["profit_percent"] >= 1.0 for o in opps))
        self.assertGreaterEqual(len(opps), 2)
        # Check descending profit order
        profits = [o["profit_percent"] for o in opps]
        self.assertEqual(profits, sorted(profits, reverse=True))

    def test_missing_price_mapping_is_skipped(self):
        streams = {
            "exchange_A": "a-b-c|",
            "exchange_B": "a-x-c|",
            "exchange_C": "a-b-c|"
        }
        price_map = {
            "a": 50000, "b": 50500, "c": 51000  # 'x' is missing
        }
        result = detect_arbitrage(streams, price_map)

        # Should ignore 'x' and not crash
        self.assertIn("x", streams["exchange_B"])
        self.assertIsInstance(result, dict)
        self.assertTrue("opportunities" in result)

    def test_custom_profit_threshold(self):
        streams = {
            "exchange_A": "a-b-c|",
            "exchange_B": "a-b-d|",
            "exchange_C": "a-b-e|"
        }
        price_map = {
            "a": 50000, "b": 50500, "c": 50800,
            "d": 50900, "e": 50950
        }
        # With threshold 0.2%, only one or two opportunities
        result = detect_arbitrage(streams, price_map, profit_threshold=0.2)
        self.assertGreaterEqual(len(result["opportunities"]), 1)

        # With threshold 2%, none should qualify
        result_high = detect_arbitrage(streams, price_map, profit_threshold=2.0)
        self.assertEqual(result_high["opportunities"], [])

    def test_stream_completion_limits_processing(self):
        streams = {
            "exchange_A": "a-b-c-d|",
            "exchange_B": "a-b-c-d|",
            "exchange_C": "a-b|"
        }
        price_map = {
            "a": 50000, "b": 50100, "c": 51000,
            "d": 52000
        }
        result = detect_arbitrage(streams, price_map)

        # Since exchange_C ends early, processing should stop at 200ms
        self.assertEqual(result["marble_diagrams"]["arbitrage"].count("O"), 0)
        self.assertTrue(result["marble_diagrams"]["arbitrage"].endswith("|"))

    def test_invalid_stream_lengths_raise_error(self):
        streams = {
            "exchange_A": "a-b|",
            "exchange_B": "a-b|"
            # Only 2 exchanges
        }
        price_map = {
            "a": 50000, "b": 50100
        }
        with self.assertRaises(ValueError):
            detect_arbitrage(streams, price_map)

    def test_max_stream_length(self):
        # 50 characters (~5 seconds)
        long_stream = "-".join(["a"] * 25) + "|"
        streams = {
            "exchange_A": long_stream,
            "exchange_B": long_stream,
            "exchange_C": long_stream
        }
        price_map = {"a": 50000}
        result = detect_arbitrage(streams, price_map)
        self.assertEqual(len(result["marble_diagrams"]["exchange_A"]), 50)
        self.assertEqual(result["opportunities"], [])

    def test_opportunity_exactly_at_1_percent(self):
        streams = {
            "exchange_A": "a--|",
            "exchange_B": "b--|",
            "exchange_C": "c--|"
        }
        price_map = {
            "a": 50000,
            "b": 50500,  # exactly 1%
            "c": 50200
        }
        result = detect_arbitrage(streams, price_map, profit_threshold=1.0)
        self.assertTrue(any(o["profit_percent"] >= 1.0 for o in result["opportunities"]))

    def test_empty_streams_return_empty_results(self):
        streams = {
            "exchange_A": "|",
            "exchange_B": "|",
            "exchange_C": "|"
        }
        price_map = {}
        result = detect_arbitrage(streams, price_map)
        self.assertEqual(result["opportunities"], [])
        self.assertEqual(result["marble_diagrams"]["arbitrage"], "|")

# Model Breaking Proof

#### Model Breaking Task URL: <https://rlhf-v3.turing.com/prompt/9472290c-5011-44a8-8166-0bba44f23689?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67648%2Fview>

#### Model code:

```python
# code generated by the model

import rx
from rx import operators as ops
from typing import Dict, List, Any

def create_price_stream(stream: str, price_map: Dict[str, float]):
    """Convert a marble string to a RxPY observable stream of prices."""
    events = []
    time = 0
    for char in stream:
        if char == '-':
            time += 100  # 100ms
        elif char == '|':
            break
        else:
            if char in price_map:
                events.append((time, price_map[char]))  # Event with timestamp and price
                # <Issue>: Increments time for price events, should only increment for '-' characters
            time += 100

    return rx.from_(events)

def detect_arbitrage(
        streams: Dict[str, str],  # Exchange name -> marble string
        price_map: Dict[str, float],  # Character -> USD price  
        profit_threshold: float = 1.0  # Minimum profit % (default 1%)
) -> Dict[str, Any]:
    # Create price streams
    stream_A = create_price_stream(streams['exchange_A'], price_map)
    stream_B = create_price_stream(streams['exchange_B'], price_map)
    stream_C = create_price_stream(streams['exchange_C'], price_map)

    # Combine the streams
    combined = rx.combine_latest(
        stream_A.pipe(ops.map(lambda e: ('exchange_A', e))),
        stream_B.pipe(ops.map(lambda e: ('exchange_B', e))),
        stream_C.pipe(ops.map(lambda e: ('exchange_C', e)))
    )

    opportunities = []
    arbitrage_events = []

    def check_arbitrage(latest_prices):
        nonlocal opportunities, arbitrage_events
        timestamp = max(price_event[1][0] for price_event in latest_prices)
        prices = {ex: price_event[1][1] for ex, price_event in latest_prices}

        # Check pairs for arbitrage
        for ex1, ex2 in [('exchange_A', 'exchange_B'), ('exchange_A', 'exchange_C'), ('exchange_B', 'exchange_C')]:
            buy_ex, sell_ex = ex1, ex2
            buy_price, sell_price = prices[buy_ex], prices[sell_ex]
            if sell_price > buy_price:
                profit_percent = ((sell_price - buy_price) / buy_price) * 100
                if profit_percent >= profit_threshold:
                    opportunities.append({
                        "timestamp": timestamp,
                        "buy_exchange": buy_ex,
                        "sell_exchange": sell_ex,
                        "buy_price": buy_price,
                        "sell_price": sell_price,
                        "profit_percent": round(profit_percent, 2)
                    })
                    arbitrage_events.append((timestamp, 'O'))  # O for opportunity

        # Sort opportunities by profit percentage, descending
        opportunities.sort(key=lambda opp: opp['profit_percent'], reverse=True)
    
    combined.subscribe(check_arbitrage)
    
    # Generate output marble diagrams
    arb_marble = ""
    current_time = 0
    max_time = max(len(s.replace('-', '')) for s in streams.values()) * 100  # based on the longest stream, assume 100ms per character
    arb_idx = 0
    while current_time < max_time and arb_idx < len(arbitrage_events):
        if arbitrage_events and arbitrage_events[arb_idx][0] == current_time:
            arb_marble += arbitrage_events[arb_idx][1]
            arb_idx += 1
        else:
            arb_marble += '-'
        current_time += 100
    arb_marble += '|'

    return {
        "opportunities": opportunities,
        "marble_diagrams": {
            "exchange_A": streams['exchange_A'],
            "exchange_B": streams['exchange_B'],
            "exchange_C": streams['exchange_C'],
            "arbitrage": arb_marble  # Marble diagram for arbitrage events
        }
    }

```